In [4]:
import os
import zipfile

local_zip = 'ia-images.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp/')
zip_ref.close()


In [5]:
alongamento_dir = os.path.join('/tmp/ia-images/ALONGAMENTO')

print('total training images:', len(os.listdir(alongamento_dir)))

alongamento_files = os.listdir(alongamento_dir)
print(alongamento_files[:10])

total training images: 19
['a-cheerful-asian-female-runner-in-sports-outfits-doing-stretchin.jpg', 'beautiful-woman-doing-yoga-exercises-at-home.jpg', 'exhausted-freelancer-woman-streching-while-working-on-a-deadline-late-at-night-employee-using-modern-technology-at-midnight-doing-overtime-for-job-business-busy-career-network-lifestyle-wireles.jpg', 'front-view-woman-stretching-at-beach.jpg', 'girl-with-earphones-standing-stretching-raising-hands-in-lock.jpg', 'healthy-lifestyle-concept-sporty-darkskinned-boy-in-black-sportswear-and-headphones-standing-on-wooden-platform.jpg', 'medium-shot-smiley-woman-stretching-outdoors.jpg', 'relaxed-woman-breathes-deeply-meditates-outdoors-keeps-arms-raised-up-keeps-eyes-closed-dressed-in-sportswear-exercises-in-urban-setting-stretches-outdoors-people-wellness-and-physical-activity.jpg', 'smiling-beautiful-asian-girl-in-pajamas-stretching-with-satisfied-face-yawning-with-closed-eyes-had-good-sleep-or-nap-waking-up-morning-woman-in-jammies-enjoying-

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

pic_index = 2

next_alongamento = [os.path.join(alongamento_dir, fname) 
                for fname in alongamento_files[pic_index-2:pic_index]]

for i, img_path in enumerate(next_alongamento):
  #print(img_path)
  img = mpimg.imread(img_path)
  plt.imshow(img)
  plt.axis('Off')
  plt.show()

In [7]:
import tensorflow as tf
import keras_preprocessing
from keras_preprocessing import image
from keras_preprocessing.image import ImageDataGenerator

TRAINING_DIR = "/tmp/ia-images/"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	    rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# VALIDATION_DIR = "/tmp/rps-test-set/"
# validation_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = training_datagen.flow_from_directory(
	TRAINING_DIR,
	target_size=(150,150),
	class_mode='categorical',
  batch_size=126
)

# validation_generator = validation_datagen.flow_from_directory(
# 	VALIDATION_DIR,
# 	target_size=(150,150),
# 	class_mode='categorical',
#   batch_size=126
# )

model = tf.keras.models.Sequential([
    # Note the input shape is the desired size of the image 150x150 with 3 bytes color
    # This is the first convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    # The second convolution
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The third convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # The fourth convolution
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the results to feed into a DNN
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    # 512 neuron hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])


model.summary()

model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

# history = model.fit(train_generator, epochs=25, steps_per_epoch=20, validation_data = validation_generator, verbose = 1, validation_steps=3)

model.save("rps.h5")


Found 19 images belonging to 1 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2

In [8]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

NameError: name 'history' is not defined

In [1]:
import tensorflow as tf 
print('tensorflow version', tf.__version__)
 
x = [[3.]]
y = [[4.]]
print('Result: {}'.format(tf.matmul(x, y)))

tensorflow version 2.10.0
Result: [[12.]]
